1. 네이버 영화 리뷰 데이터셋을 활용하여, 자연어 모델링을 위한 형태소 분석 등 전처리 기법을 학습합니다.

2. 전처리된 자연어 데이터를 벡터화하는 TF-IDF 기법을 이해합니다.

3. 벡터화된 데이터를 활용하여 토픽 모델링과 감성분석 모델링을 실습해봅니다.

In [10]:
# (구글 코랩에서 실습할 경우) 아래 코드를 실행하여 한글 폰트를 설치합니다. 
# 설치가 완료되면, 커널을 재시작합니다. (상단 메뉴에서 [런타임] - [런타임 다시 시작] 을 선택)

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (7,779 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

# 라이브러리 불러오기


In [11]:
import pandas as pd
import numpy as np

import os, re
from tqdm import tqdm

# 경고문구 미표시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

# 네이버 영화 리뷰 데이터 - 다운로드
-  출처: https://github.com/e9t/nsmc

In [13]:
# 다운로드 받을 폴더를 준비
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

In [15]:
# 데이터 다운로드 
import torchtext 
torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', 
                                  path=os.path.join(DATA_DIR, 'review.txt'))

14.6MB [00:00, 19.3MB/s]


'/content/data/review.txt'

In [16]:
# txt 파일을 판다스 데이터프레임으로 읽어오기
data = pd.read_csv(os.path.join(DATA_DIR, 'review.txt'), sep='\t')

# 데이터프레임의 크기
print(data.shape)

# 첫 5행을 출력
data.head()

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 한글 형태소 분석

한국어 정보처리를 위한 파이썬 패키지인 KoNLPy(코엔엘파이)를 활용하여

분석 대상이 되는 문장(텍스트)를 최소 의미 단어를 뜻하는 형태소를 기준으로 토큰(token)으로 분리

## **1) KoNLPy 설치**

In [17]:
# 코랩에 konlpy 패키지를 설치합니다.
! pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 6.1 MB/s eta 0:00:00


## **2) KoNLPy 한글 형태소 분석**

형태소 분석기의 한 종류인 Okt tagger의 다양한 분석 함수를 적용하고, 각 함수의 기능과 차이를 이해합니다.

In [18]:
# 긍정 리뷰를 하나 선택
sample_text = data['document'].iloc[149978]
print(type(sample_text))
print(sample_text)

<class 'str'>
그리 만족스럽진못했어도 7점은 나와야되는것같아 10점줌. 주인공들연기도 훌륭했고 내용도 이정도면 괜찮았다. 해피엔딩으로끝났으면 그저 진부한영화가 되버릴뻔. 그래도 엔딩이 섭섭한건 없지않아있었음. 킬러들의도시도그렇고 콜린파렐을 너무 축축하게 끝내버리는듯


In [19]:
# 트위터 형태소 분석기(Okt)를 활용
from konlpy.utils import pprint
from konlpy.tag import Okt
okt = Okt()
print(okt)

In [20]:
# (단어, 품사) 추출
tokens = okt.pos(sample_text)
pprint(tokens)

[('그리', 'Verb'),
 ('만족스럽진못', 'Adjective'),
 ('했어도', 'Verb'),
 ('7', 'Number'),
 ('점', 'Noun'),
 ('은', 'Josa'),
 ('나와야', 'Verb'),
 ('되는것', 'Verb'),
 ('같아', 'Adjective'),
 ('10', 'Number'),
 ('점줌', 'Noun'),
 ('.', 'Punctuation'),
 ('주인공', 'Noun'),
 ('들', 'Suffix'),
 ('연', 'Modifier'),
 ('기도', 'Noun'),
 ('훌륭했고', 'Adjective'),
 ('내용', 'Noun'),
 ('도', 'Josa'),
 ('이', 'Determiner'),
 ('정도', 'Noun'),
 ('면', 'Josa'),
 ('괜찮았다', 'Adjective'),
 ('.', 'Punctuation'),
 ('해피엔딩', 'Noun'),
 ('으로', 'Josa'),
 ('끝났으면', 'Verb'),
 ('그저', 'Adverb'),
 ('진부한', 'Adjective'),
 ('영화', 'Noun'),
 ('가', 'Josa'),
 ('되', 'Verb'),
 ('버릴', 'Verb'),
 ('뻔', 'Noun'),
 ('.', 'Punctuation'),
 ('그래도', 'Adverb'),
 ('엔딩', 'Noun'),
 ('이', 'Josa'),
 ('섭섭한건', 'Adjective'),
 ('없지', 'Adjective'),
 ('않아있었음', 'Verb'),
 ('.', 'Punctuation'),
 ('킬러', 'Noun'),
 ('들', 'Suffix'),
 ('의', 'Josa'),
 ('도시', 'Noun'),
 ('도', 'Josa'),
 ('그렇고', 'Adjective'),
 ('콜린파렐', 'Noun'),
 ('을', 'Josa'),
 ('너무', 'Adverb'),
 ('축축하게', 'Adjective'),
 ('끝내', 'No

In [21]:
# 단어 ONLY 추출
tokens = okt.morphs(sample_text)
pprint(tokens)

['그리',
 '만족스럽진못',
 '했어도',
 '7',
 '점',
 '은',
 '나와야',
 '되는것',
 '같아',
 '10',
 '점줌',
 '.',
 '주인공',
 '들',
 '연',
 '기도',
 '훌륭했고',
 '내용',
 '도',
 '이',
 '정도',
 '면',
 '괜찮았다',
 '.',
 '해피엔딩',
 '으로',
 '끝났으면',
 '그저',
 '진부한',
 '영화',
 '가',
 '되',
 '버릴',
 '뻔',
 '.',
 '그래도',
 '엔딩',
 '이',
 '섭섭한건',
 '없지',
 '않아있었음',
 '.',
 '킬러',
 '들',
 '의',
 '도시',
 '도',
 '그렇고',
 '콜린파렐',
 '을',
 '너무',
 '축축하게',
 '끝내',
 '버리는듯']


In [22]:
# 명사 ONLY 추출
tokens = okt.nouns(sample_text)
pprint(tokens)

['점',
 '점줌',
 '주인공',
 '기도',
 '내용',
 '정도',
 '해피엔딩',
 '영화',
 '뻔',
 '엔딩',
 '킬러',
 '도시',
 '콜린파렐',
 '끝내']


## **3) 텍스트 전처리**

Okt 분석 도구 중에서 명사를 추출하는 함수를 사용하여, 각 리뷰 문장에서 명사를 추출하여 토큰화합니다. 이 때, 두 글자 이하의 단어를 분석대상에서 제외하기로 합니다.

In [23]:
# 누락 데이터를 제거
review_data = data['document'].dropna().values

# 학습 속도를 고려하여, 5000개의 샘플을 선택하여 추출
review_data = review_data[:5000]

# 배열의 크기
print(review_data.shape)

# 첫번째 데이터
print(review_data[0])

# 마지막 데이터
print(review_data[4999])

(5000,)
아 더빙.. 진짜 짜증나네요 목소리
저두 별주기가 아까움..무슨 내용인지..쩝,ㅠ.ㅠ


In [24]:
# 세 글자 이상의 명사를 사용 (두 글자 이하의 단어는 제거)
cleaned_review_data = []

for review in tqdm(review_data):
    tokens = okt.nouns(review)
    cleaned_tokens = []
    for word in tokens:
        if len(word) > 2:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)
    cleaned_review_data.append(cleaned_review)

print(len(cleaned_review_data))
print(cleaned_review_data[0])

100%|██████████| 5000/5000 [00:29<00:00, 167.00it/s]

5000
목소리


## 4**) TF-IDF 벡터로 표현**

LDA, 로지스틱 회귀 등 머신러닝의 입력값은 수치형태로 표현된 매트릭스입니다. 머신러닝 모델을 활용하기 위해 이 프로젝트에서는 TF-IDF의 형태로 텍스트 문서를 수치화 합니다.


In [25]:
# 사이킷런 패키지 활용 
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 변환기 객체를 생성
tfid = TfidfVectorizer()

# TF-IDF 변환기에 데이터를 입력하여 변환
review_tfid = tfid.fit_transform(cleaned_review_data)

# 배열의 크기
print(review_tfid.shape)

# 첫 번째 데이터
print(review_tfid[0])

(5000, 2138)
  (0, 574)	1.0


In [26]:
# 단어 사전 확인 (딕셔너리 형태)
vocab = tfid.vocabulary_

# 단어 사전의 크기
print(len(vocab))

# 단어 사전 출력 (앞에서 5개의 단어만 출력)
print({ k:v for i, (k, v) in enumerate(vocab.items()) if i < 5 })

2138
{'목소리': 574, '포스터': 1958, '교도소': 120, '이야기': 1422, '스파이더맨': 978}


In [27]:
# 단어들의 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
index_to_word = { v:k for k, v in vocab.items() } 

# 앞에서 5개의 단어를 출력
print({  k:v for i, (k, v) in enumerate(index_to_word.items()) if i < 5 })

{574: '목소리', 1958: '포스터', 120: '교도소', 1422: '이야기', 978: '스파이더맨'}


In [28]:
# 첫 번째 리뷰를 구성하는 단어들의 사전 인덱스를 이용하여 원래 단어를 복원 (순서 복원 X)
original_text = " ".join([index_to_word[word_idx] for word_idx in review_tfid[0].indices])
original_text

'목소리'

# **Topic Modeling**

문서는 여러 종류의 단어들의 집합체로 구성됩니다. 문서를 구성하는 단어들의 확률분포를 활용하여 문서의 토픽(주제)를 파악할 수 있습니다


## **1) LDA (LatentDirichletAllocation)**

LDA 토픽 모델링을 이용하여, 문서로부터 두 종류의 토픽을 추출합니다. 문서의 감성(긍정, 부정) 형태로 토픽이 추출되는지, 아니면 다른 종류의 토픽이 추출되는지 분석하세요.


In [29]:
# 사이킷런 패키지 활용
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델링 객체를 생성 (토픽 갯수를 2로 설정: 긍정/부정)
lda = LatentDirichletAllocation(n_components=2)  

# TF-IDF 벡터를 입력하여 모델 학습 
lda.fit(review_tfid)

LatentDirichletAllocation(n_components=2)

In [30]:
# 토픽 모델링 결과를 담고 있는 배열의 형태 : (2개의 토픽, 2157개의 단어)
print(lda.components_.shape)

(2, 2138)


In [31]:
# 2157개의 단어 중에서, 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력

for idx, topic in enumerate(lda.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

토픽 유형 1: [('드라마', 98.809), ('쓰레기', 68.094), ('주인공', 60.34), ('마지막', 54.333), ('이야기', 47.965), ('연기력', 20.031), ('최고다', 19.919), ('긴장감', 17.273), ('로맨스', 16.888), ('공포영화', 16.843)]
토픽 유형 2: [('스토리', 103.265), ('이영화', 38.382), ('코미디', 24.195), ('시리즈', 21.756), ('캐릭터', 20.757), ('캐스팅', 19.399), ('스릴러', 19.03), ('우리나라', 18.271), ('한국영', 18.069), ('포스터', 17.51)]


## **2) pyLDAvis 시각화**

pyLDAvis 패키지를 활용하여, LDA 토픽 모델링 경과를 시각화 합니다.

In [32]:
# pyLDAvis 설치
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=837230f67db8a1314c48cbdcf32071650c4c1b8c6f1e83e3739c00ae65ae96b2
  Stored in directory: /root/.cache/pip/wheels/90/61/ec/9dbe9efc3acf9c4e37ba70fbbcc3f3a0ebd121060aa593181a
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=4ab7cf2aac74adf366c211189e457d3988de07a057c07ab7a4d426686e3c5aee
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built pyLDAvis sklearn


In [33]:
# LDA 토픽 모델링 결과를 시각화
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda, review_tfid, tfid)
pyLDAvis.display(visualization)

/usr/local/lib/python3.8/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/usr/local/lib/python3.8/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping


## **3) 추가 LDA** (토픽 3개)




In [43]:
# 사이킷런 패키지 활용
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델링 객체를 생성 (토픽 갯수를 3으로 설정)
lda2 = LatentDirichletAllocation(n_components=3)  

# TF-IDF 벡터를 입력하여 모델 학습 
lda2.fit(review_tfid)

LatentDirichletAllocation(n_components=3)

In [44]:
# 토픽 모델링 결과를 담고 있는 배열의 형태 : (3개의 토픽)
print(lda2.components_.shape)

(3, 2138)


In [46]:
# 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력

for idx, topic in enumerate(lda2.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

토픽 유형 1: [('스토리', 103.155), ('드라마', 98.649), ('쓰레기', 67.951), ('이야기', 47.874), ('최고다', 19.786), ('스릴러', 18.901), ('한국영', 17.928), ('캐릭터', 15.872), ('갈수록', 15.209), ('도대체', 14.622)]
토픽 유형 2: [('주인공', 60.23), ('마지막', 54.229), ('연기력', 21.461), ('캐스팅', 19.251), ('로맨스', 16.779), ('영화관', 15.647), ('그대로', 12.797), ('무조건', 12.164), ('극장판', 10.731), ('프랑스', 10.321)]
토픽 유형 3: [('이영화', 38.229), ('코미디', 24.061), ('시리즈', 21.619), ('우리나라', 18.116), ('포스터', 17.362), ('공포영화', 16.713), ('한마디', 16.663), ('제대로', 15.596), ('영화인', 13.987), ('몰입도', 13.817)]


In [47]:
# LDA2 토픽 모델링 결과를 시각화
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda2, review_tfid, tfid)
pyLDAvis.display(visualization)

# **Sentimental Analysis (감성 분석)**

감성 분석은 문장(텍스트)에 드러나는 사람들의 생각이나 느낌, 성향과 같이 주관적인 감성을 분석하는 자연어 처리 기법입니다. 머신러닝 분류 모델을 활용하여 문장이 갖는 긍정 또는 부정의 감성을 예측하세요.

## 1) 머신러닝 알고리즘을 활용하여, 긍정/부정 감성을 분류

In [34]:
labels = data['label'].iloc[:5000].values
print(labels.shape)
print(labels[:5])

(5000,)
[0 1 0 0 1]


In [35]:
# 사이킷런 패키지 활용
from sklearn.linear_model import LogisticRegression

# 로지스틱 분류 모델링 객체를 생성 
lr = LogisticRegression()

# TF-IDF 벡터를 입력하여 모델 학습 
lr.fit(review_tfid, labels)

LogisticRegression()

In [36]:
# 첫 번째 샘플을 이용하여 모델 예측
pred = lr.predict(review_tfid[0])
print(pred)

[0]


## 2) Part 1에서 수집한 리뷰를 긍정, 부정으로 감성 분류 예측


In [37]:
# 판다스 데이터프레임로 읽어오기
test = pd.read_csv('/content/drive/MyDrive/HUFS Data Analysis/naver_review_Avatar.csv')

test.head()

,Unnamed: 0,title,user,review
0,0,"<아바타 2> 주인공 ""네이티리"" 캐릭터 컨셉 아트 이미지 대공개...",insu****,"\n\n\n\n\n아바타 2\n\n감독\n제임스 카메론\n출연\n시고니 위버, 스티..."
1,1,2014년에 이글을 읽는사람들에게..,xnzm****,"\n2014년에 아바타2가 개봉하다니ㅋ\n \n아바타1,, 정말 재밌게 봤었는데\n..."
2,2,"[필독, 스압] 위대한 지휘관, 쿼리치대령을 추모하며...",alwa****,\n영화 아바타는.끝까지 우리 인류의 번영과 이익을 위해 사투를 벌이다 죽어간 위대...
3,3,"아바타2 3,4 제작소식 모음",soqu****,\n \n \n \n<< 아바타 속편 제작소식 모음 >>\n \n\n \n \n...
4,4,아바타2 스토리,doub****,\n전작의 경우 cg는 혁명적이었지만제임스 카메론이 14년동안 구상한거 치곤 스토리...


In [38]:
# 첫 번째 리뷰를 선택
test_sample = test['review'][0]
test_sample

'\n\n\n\n\n아바타 2\n\n감독\n제임스 카메론\n출연\n시고니 위버, 스티븐 랭, 조 샐다나, 샘 워싱턴\n개봉\n2017 미국\n\n리뷰보기\n\n\n\u200b*** 아바타 2 (2017) ***장르: Sci-Fi, 판타지, 모험, 액션감독: 제임스 카메론주연: 조 셀다나 (네이티리 역), 샘 워싱턴 (제이크 설리 역), 시고니 위버 (그레이스 어거스틴 역), 스티븐 랭 (마일즈 쿼리치 역)...국적: 미국북미 개봉일: 2017년 12월 25일2017년 12월 25일 북미 개봉작으로 기대를 모으고 있는 영화 아바타 2...특히나 주인공 "네이티리" 의 캐릭터 이미지가 공개가 되어서 올려봅니다.배우 조 샐다나와 샘 워싱턴, 시고니 위버 그리고 스티븐 랭까지 전편에서 출연했었던 배우들이 그대로 출연을 하고 있어서더욱 기대가 되는 영화인 것 같습니다. 아무튼 제임스 카메론 감독의 영화 아바타 2편을 시작으로2018년 아바타 3와 2019년 아바타 4까지 진행되고 있어서 앞으로 더욱 기대되는 영화 아바타인 것 같습니다.\n'

In [39]:
# 한글을 제외하고 전부 제거
test_sample = re.sub(r"[^가-힣]", "", test_sample)
test_sample

'아바타감독제임스카메론출연시고니위버스티븐랭조샐다나샘워싱턴개봉미국리뷰보기아바타장르판타지모험액션감독제임스카메론주연조셀다나네이티리역샘워싱턴제이크설리역시고니위버그레이스어거스틴역스티븐랭마일즈쿼리치역국적미국북미개봉일년월일년월일북미개봉작으로기대를모으고있는영화아바타특히나주인공네이티리의캐릭터이미지가공개가되어서올려봅니다배우조샐다나와샘워싱턴시고니위버그리고스티븐랭까지전편에서출연했었던배우들이그대로출연을하고있어서더욱기대가되는영화인것같습니다아무튼제임스카메론감독의영화아바타편을시작으로년아바타와년아바타까지진행되고있어서앞으로더욱기대되는영화아바타인것같습니다'

In [40]:
# 세 글자 이상의 명사만을 추출

tokens = okt.nouns(test_sample)
cleaned_tokens = []
for word in tokens:
    if len(word) > 2:
        cleaned_tokens.append(word)
    else:
        pass
cleaned_review = " ".join(cleaned_tokens)

cleaned_review

'아바타 제임스 카메론 워싱턴 아바타 판타지 제임스 카메론 네이티 리역샘 워싱턴 제이크 레이스 어거스틴 스티븐 개봉일 아바타 네이티 캐릭터 이미지 워싱턴 스티븐 영화인 제임스 카메론 아바타 아바타 아바타 아바타'

In [41]:
# TF-IDF 변환기에 데이터를 입력하여 변환
test_review_tfid = tfid.transform([cleaned_review])

# 배열을 출력
print(test_review_tfid)

  (0, 1923)	0.10609705587931596
  (0, 1813)	0.08608327258267265
  (0, 1631)	0.38929373164534287
  (0, 1398)	0.12380202717019935
  (0, 1243)	0.09665830062936952
  (0, 1065)	0.8666141901913955
  (0, 974)	0.23258020501269236


In [42]:
# 로지스틱 회귀 모델을 활용하여 분류 예측
test_pred = lr.predict(test_review_tfid)[0]
print("분석 결과 {}적인 리뷰로 예측됩니다. ".format("긍정" if test_pred > 0 else "부정"))

분석 결과 부정적인 리뷰로 예측됩니다. 
